# Docker commands

Here we describe those docker commands that do not merit a separate discussion.

# Working with containers

## `ps` - show containers

To view active containers, you should use the `docker ps` command.

In [4]:
%%bash
docker run -itd --rm --name test_container python:3.10 &> /dev/null
docker ps
docker stop test_container &> /dev/null

CONTAINER ID   IMAGE         COMMAND     CREATED        STATUS                  PORTS     NAMES
b17984f2b882   python:3.10   "python3"   1 second ago   Up Less than a second             test_container


### `-a` - show all

By default, inactive containers are hidden.

In [8]:
%%bash
docker run --name temp_container ubuntu &> /dev/null
echo "========no option -a=============="
docker ps
echo "=========with option -a============="
docker ps -a
docker rm temp_container &> /dev/null

========no option -a==============
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
=========with option -a=============
CONTAINER ID   IMAGE     COMMAND       CREATED                  STATUS                              PORTS     NAMES
ad0439c23847   ubuntu    "/bin/bash"   Less than a second ago   Exited (0) Less than a second ago             temp_container


## `history` - show layers of the image

## `start` - activate exited container

In the next cell, I build a ubuntu container that prints "hello world" when it starts. Like all standard ubuntu containers it quits, which we can see in the `STATUS` of the `ps` command. Then by `docker start` the container is reactivated.

For some reason it only works when I use the `-i` option for `docker start`.

In [16]:
%%bash
cd containers_files
docker build -t test_ubuntu . &> /dev/null
echo "=====First start====="
docker run --name u1 test_ubuntu
echo "=====List of all containers====="
docker ps -a
echo "=====Restart====="
docker start -i u1
docker rm u1 &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====First start=====
hello world
=====List of all containers=====
CONTAINER ID   IMAGE         COMMAND                CREATED        STATUS                              PORTS     NAMES
af533b577c94   test_ubuntu   "echo 'hello world'"   1 second ago   Exited (0) Less than a second ago             u1
=====Restart=====
hello world


## `stop` - stop some container

In the following example, I start a container with Ubuntu in such a way that it stays running, which was proven by using `docker ps`. But after using `docker stop` the same container is in the state "Exited".

In [20]:
%%bash
docker run -itd --name test_ubuntu ubuntu &> /dev/null
echo "=====Just started container====="
docker ps -a
docker stop test_ubuntu &> /dev/null
echo "=====Stoped container (STATUS \"Exited\")====="
docker ps -a
docker rm test_ubuntu &> /dev/null

=====Just started container=====
CONTAINER ID   IMAGE     COMMAND       CREATED                  STATUS                  PORTS     NAMES
e175ae336dcb   ubuntu    "/bin/bash"   Less than a second ago   Up Less than a second             test_ubuntu
=====Stoped container (STATUS "Exited")=====
CONTAINER ID   IMAGE     COMMAND       CREATED          STATUS                                PORTS     NAMES
e175ae336dcb   ubuntu    "/bin/bash"   10 seconds ago   Exited (137) Less than a second ago             test_ubuntu


## `exec` - run in the container

In the next command I start ubuntu conteiner. Then using `docker exec' I run the `ls' command in the container and get a typical Linux result for that command.

In [24]:
%%bash
docker run -itd --rm --name test_ubuntu ubuntu &> /dev/null
docker exec test_ubuntu ls
docker stop test_ubuntu &> /dev/null

bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var


### `-it` - take control

It looks like the options individually have the same properties as in the `docker run` command. But the important thing is that you can take control of the detached container with this command.<br> 
The following example shows how I can get control in detached ubuntu by running the `bash` command with named options.

<img src="containers_files/docker exec -it.gif"></img>

## `logs` - get container console

Desrtiption of the command you can find  [here](logs.ipynb#sec-docker_logs_command).

# `run` - container from image

This command is related to the "Working with containers" section, but it is big enough and important enough to have its own section.

In [9]:
%%bash
echo "================before start====================="
docker ps -a
echo "===============after start======================"
docker run --name temp_container ubuntu &> /dev/null
docker ps -a
docker rm temp_container

================before start=====================
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
===============after start======================
CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                              PORTS     NAMES
cd0f144063c5   ubuntu    "/bin/bash"   1 second ago   Exited (0) Less than a second ago             temp_container
temp_container


## `-rm` - automatic container removal

After stopping, the container is removed. In the following example there are two containers of ubuntu, one with the `--rm` option and the other without. Both containers stops by default - but only one stays with `STATUS == Exited`.

In [12]:
%%bash
docker run --name no_rm_ubuntu ubuntu
docker run --rm --name ubuntu_with_rm ubuntu
docker ps -a
docker rm no_rm_ubuntu &> /dev/null

CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                              PORTS     NAMES
7acc3cc0c61c   ubuntu    "/bin/bash"   1 second ago   Exited (0) Less than a second ago             no_rm_ubuntu


## `--name` - set name for the container

In [2]:
%%bash
docker run --rm --name unbuntu_container -d ubuntu
docker ps
docker stop unbuntu_container

7116dcdaa0a131a8507f9cbe149719498456fe356a23f1ddfc48f65080cfe660
CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                  PORTS     NAMES
7116dcdaa0a1   ubuntu    "/bin/bash"   1 second ago   Up Less than a second             unbuntu_container
unbuntu_container


## `-d` - make container not to lock CLI

In [15]:
%%bash
docker run --rm -d --name temp_nginx nginx &> /dev/null
docker ps
docker stop temp_nginx &> /dev/null

CONTAINER ID   IMAGE     COMMAND                  CREATED                  STATUS                  PORTS     NAMES
09175cb8c6d9   nginx     "/docker-entrypoint.…"   Less than a second ago   Up Less than a second   80/tcp    temp_nginx


## `-i` - interactive mode

Mode that allows to interapt with container.

Unfortunately it is not possible to show how this mode will work in jupyter. So here is a gif of me experimenting with the following command:

`docker run -i --rm --name test_ubuntu ubuntu`

<img src="containers_files/docker -i option.gif"></img>

In the example, the ubuntu container has been started and is stopped by default. The `-i` option causes it to lock the terminal. Extremely interesting that you will be able to send messages to the container and get results (as I showed with `ls` and `exit` commands), but you won't get the description of the user common for Linux terminal.

## `-t` - turn on TTY

TTY is something on backend language. The point here is that in combination with the `-i` option you can make will show you the "description" of the user which was mentioned in the previous section. And the terminal will be blocked waiting for input. But this option don't allow you to send anything to the container.

The following `.gif` show what you will get in case using command `docker run -t --rm --name temp_ubuntu ubuntu`.

<img src="containers_files/docker -t option.gif"></img>

## `-it` - combination

Both options were discovered above, but they are really often used in combination. So Ubuntu provides full CLI interface with this options.

<img src="containers_files/docker -it option.png"></img>

## Commands from jupyter

Using only the `i` option (without `t`), you can send commands to the container from the Jupyter cell.

So in the following example, I can easily run the `ls` program from the newly created ubuntu container.

In [2]:
%%bash
docker run --rm -i --name test_container ubuntu
ls
exit

bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var


It's a pity I didn't understand this before I wrote most of the Docker description. So don't be surprised if you see something like this: 
<div class="sourceCode">
<!--This class is needed to display as code on site-->
docker exec ...<br>
docker exec ...<br>
docker exec ...<br>
</div>
In some section of docker description.

## `-e` - environment variables

You can also pass the values of the variables of the system you are calling from.

In [7]:
%%bash
docker run --rm -i --name e_example -e FEDOR_TEST_VAR=$HOME ubuntu
echo $FEDOR_TEST_VAR
exit

/home/fedor


## Command on container start{#sec-docker_run_com_on_start}

In the `docker run` command, after the name of the base image, you can specify a command that will be executed when the container starts. So in the next example I will show you how to run `echo "hello world"` in a new ubuntu container.

In [2]:
%%bash
docker run --rm ubuntu echo "hello world"

hello world


- If you want to run several commands you have to use `bash -c "<command>"`, like in following example:

In [5]:
%%bash
docker run --rm ubuntu bash -c "
    echo \"hello world\"; 
    echo \"hello world2\"
"

hello world
hello world2


- Note that when containers are in interactive mode, the command executed should take input, otherwise it will just leave container anyway, the follwing gif describe how it looks like:
<img src="containers_files/docker command -it option.gif"></img>


In the example, I ran the `echo` command with container starts and showed that there were no containers created - because the container was stopped and removed just after it was started. But immediately I created a new container, but now with the command `bash -c "echo \"hello world\"; bash` - in this case the last command is `bash` which waits for input, so I'm comming back into the container.

## `-v` - map folder on container to folder on host

This is wide topic detailed in [this](docker_volumes.ipynb) section;

## `-u` - set user

By default, all docker containers have root privileges, but it can be useful to set user. For a more detailed description in the context of filesystem access, see [here](docker_volumes.ipynb#sec-run_u_option).

## `-p` - use port

For mode details see [this](ports.ipynb).

# Working with images

## `pull` - download image from dockerhub.com

It is obligatory to pass on one of the:

- `<image name>:<tag>` (`<tag>` - by default is `latest`);
- `IMAGE ID` - image id.

In [6]:
%%bash
docker images | grep hello-world
echo "===============beforre pull======================"
docker pull hello-world:latest &> /dev/null
echo "===============after pull======================"
docker images | grep hello-world
docker rmi hello-world &> /dev/null

===============beforre pull======================
===============after pull======================
hello-world                latest    9c7a54a9a43c   2 weeks ago     13.3kB


## `images` - show list of available images

In [31]:
%%bash
docker images

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
hello-world   latest    feb5d9fea6a5   16 months ago   13.3kB


### `--a`,`--all` - show all images

By default, so called "intermediate", images are hiden. Nowadays I don't know what "intermediate" images are.

In [32]:
%%bash
docker images --all

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
hello-world   latest    feb5d9fea6a5   16 months ago   13.3kB


### `--digests` - show digests

In [33]:
%%bash
docker images --digests

REPOSITORY    TAG       DIGEST                                                                    IMAGE ID       CREATED         SIZE
hello-world   latest    sha256:aa0cc8055b82dc2509bed2e19b275c8f463506616377219d9642221ab53cf9fe   feb5d9fea6a5   16 months ago   13.3kB


### `--format` - format output

Allows output to be formatted using GO templates.

In [34]:
%%bash
docker images --format '{{.Repository}}:{{.Tag}}'

hello-world:latest


### `--no-trunc` - full output

In [35]:
%%bash
docker images --no-trunc

REPOSITORY    TAG       IMAGE ID                                                                  CREATED         SIZE
hello-world   latest    sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412   16 months ago   13.3kB


### `-q`, `--quiet` - print only images ids

In [36]:
%%bash
docker images -q

feb5d9fea6a5


## `rmi` - delete images

It is obligatory to pass on one of the:

- `<image name>:<tag>` (`<tag>` - by default is `latest`);
- `IMAGE ID` - image id.

In [39]:
%%bash
docker images
echo "==========================="
docker rmi hello-world:latest
echo "==========================="
docker images

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
hello-world   latest    feb5d9fea6a5   16 months ago   13.3kB
Untagged: hello-world:latest
Untagged: hello-world@sha256:aa0cc8055b82dc2509bed2e19b275c8f463506616377219d9642221ab53cf9fe
Deleted: sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412
Deleted: sha256:e07ee1baac5fae6a26f30cabfe54a36d3402f96afda318fe0a96cec4ca393359
REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


## `build` - build image from dockerfile

This command has description [here](image_creation.ipynb#sec-docker_build_command)

## `commit` - save container state

This command has description [here](image_creation.ipynb#sec-docker_commit_command).

# `cp` - copy files from/to container

Allows files to be swapped between the host file system and the running container.

## host -> container

There is the following syntax:

`docker cp <host path> <cantainer name>:<container path>`

The following example shows how to copy a file into a container.

## container -> host

There is syntax for m

In [8]:
%%bash
cd filesystem_example
docker run --rm --name test_ubuntu -itd ubuntu &> /dev/null

docker cp copied_message.txt test_ubuntu:copied_message.txt

echo "=====copied to container====="
docker exec test_ubuntu cat copied_message.txt
echo ""

docker cp test_ubuntu:copied_message.txt new_message.txt
echo "=====copied from container====="
cat new_message.txt

rm new_message.txt
docker stop test_ubuntu &> /dev/null

=====copied to container=====
This message is for checking the copy functions.
=====copied from container=====
This message is for checking the copy functions.

# `instpect` - docker object info

## Basic examples

This command provide information about docker ojects in json format such as:

- **images**:

In [1]:
%%bash
docker inspect ubuntu

[
    {
        "Id": "sha256:74f2314a03de34a0a2d552b805411fc9553a02ea71c1291b815b2f645f565683",
        "RepoTags": [
            "ubuntu:latest"
        ],
        "RepoDigests": [
            "ubuntu@sha256:2adf22367284330af9f832ffefb717c78239f6251d9d0f58de50b86229ed1427"
        ],
        "Parent": "",
        "Comment": "",
        "Created": "2023-03-01T04:38:49.239257335Z",
        "Container": "298f60554671ae2f5bf43b9892526aaa221e8093c9cee1ca68ef65fc3ac67600",
        "ContainerConfig": {
            "Hostname": "298f60554671",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "Tty": false,
            "OpenStdin": false,
            "StdinOnce": false,
            "Env": [
                "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
            ],
            "Cmd": [
                "/bin/sh",
                "-c",
                "#(

- **containers**:

In [5]:
%%bash
docker run --rm --name test -itd ubuntu &> /dev/null
docker inspect test
docker stop test &> /dev/null

[
    {
        "Id": "020c99a8fbdca844a78956c0f7b2aba4fcd53e4f299112894dd0c0b7aca246c5",
        "Created": "2023-06-12T13:37:52.23307605Z",
        "Path": "/bin/bash",
        "Args": [],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 32562,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2023-06-12T13:37:52.471660971Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:74f2314a03de34a0a2d552b805411fc9553a02ea71c1291b815b2f645f565683",
        "ResolvConfPath": "/var/lib/docker/containers/020c99a8fbdca844a78956c0f7b2aba4fcd53e4f299112894dd0c0b7aca246c5/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/020c99a8fbdca844a78956c0f7b2aba4fcd53e4f299112894dd0c0b7aca246c5/hostname",
        "HostsPath": "/var/lib/docker/containers/020c99a8

- **networks**:

In [9]:
%%bash
docker inspect bridge

[
    {
        "Name": "bridge",
        "Id": "cf037e1d3aeb308ae603f8028da3343134d92e79462fdcb5390b196a9e49c87d",
        "Created": "2023-06-12T08:14:24.801403507+03:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {
            "com.docker.network.bridge.default_bridge": "true",
            "com.docker.network.bridge.enable_icc": "true",
            "com.docker.network.bridge.enable_ip_masquerade": "true",
            "com.docker.network.bridge.host_binding_ipv4": "0.0.0.0",
    

## **Special cases**

- For more information about the **path to container logs**, you can look [here](logs.ipynb#log_path_on_computer).